In [14]:
import torch
from diffusers import KandinskyV22PriorEmb2EmbPipeline

from models.combined_pipeline_prior import CombinedPipelineV2


In [15]:
decoder = CombinedPipelineV2.from_pretrained("../models/weights/img2img-painting", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
decoder.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

In [16]:
prior = KandinskyV22PriorEmb2EmbPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior',
                                                         image_encoder=decoder.image_encoder,
                                                         scheduler=decoder.scheduler,
                                                         image_processor=decoder.image_processor,
                                                         torch_dtype=torch.float16)
prior.enable_model_cpu_offload()

text_encoder\model.safetensors not found


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
negative_prior_prompt ='lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature'
torch.manual_seed(42)

In [13]:
def generate_with_prior(source_image, target_image):
    clip_img_emb = prior.interpolate(images_and_prompts=[source_image, target_image],
                                     weights=[1],
                                     num_images_per_prompt=1, ).image_embeds
    img_emb = prior(prompt='a human being wearing some clothes in front of a white background',
                    image=clip_img_emb,
                    strength=0.85,
                    num_inference_steps=25,
                    num_images_per_prompt=1,)
    negative_emb = prior(prompt=negative_prior_prompt,
                         image=clip_img_emb,
                         strength=1,
                         num_inference_steps=25,
                         num_images_per_prompt=1)
    images = decoder(image=img,
                     strength=0.5,
                     image_embeds=img_emb.image_embeds,
                     negative_image_embeds=negative_emb.image_embeds,
                     num_inference_steps=50,
                     height=512, width=512)


NameError: name 'img' is not defined